In [68]:
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
#from sklearn.metrics import accuracy_score, mean_squared_error

In [69]:
training_dataset = pd.read_csv("ML_Data_Set.csv")

In [80]:
def calculate_team_statistic(player_ids, stat_dict):
    valid_player_ids = [player_id for player_id in player_ids if player_id != 0]
    if valid_player_ids:
        return sum(stat_dict.get(player_id, 0) for player_id in valid_player_ids) / len(valid_player_ids)
    else:
        return 0

def teamavgs_feature(training_set):
    player_stats = {}
    df = training_set

    # Iterate over each match to collect player statistics
    for index, row in df.iterrows():
        # Extract player IDs for both teams
        team1_players = [player_id for player_id in row[['Team 1 P1', 'Team 1 P2', 'Team 1 P3', 'Team 1 P4', 'Team 1 P5', 'Team 1 P6', 'Team 1 P7', 'Team 1 P8']] if player_id != 0]
        team2_players = [player_id for player_id in row[['Team 2 P1', 'Team 2 P2', 'Team 2 P3', 'Team 2 P4', 'Team 2 P5', 'Team 2 P6', 'Team 2 P7', 'Team 2 P8']] if player_id != 0]

        for player_id in team1_players:
            player_stats[player_id] = player_stats.get(player_id, {"matches_played": 0, "matches_won": 0, "total_GS": 0, "total_GC": 0})
            player_stats[player_id]["matches_played"] += 1
            player_stats[player_id]["total_GS"] += row['Team 1 Goals']
            player_stats[player_id]["total_GC"] += row['Team 2 Goals']
            if row['Team 1 Result'] == 1:  # Team 1 won
                player_stats[player_id]["matches_won"] += 1

        for player_id in team2_players:
            player_stats[player_id] = player_stats.get(player_id, {"matches_played": 0, "matches_won": 0, "total_GS": 0, "total_GC": 0})
            player_stats[player_id]["matches_played"] += 1
            player_stats[player_id]["total_GS"] += row['Team 2 Goals']
            player_stats[player_id]["total_GC"] += row['Team 1 Goals']
            if row['Team 2 Result'] == 1:  # Team 2 won
                player_stats[player_id]["matches_won"] += 1
    
    # Calculate win percentage, average goals, and average goals conceded for each player
    win_percentage_dict = {}
    avg_GS_dict = {}
    avg_GC_dict = {}

    for player_id, stats in player_stats.items():
        win_percentage_dict[player_id] = (stats["matches_won"] / stats["matches_played"]) * 100 if stats["matches_played"] > 0 else 0
        avg_GS_dict[player_id] = stats["total_GS"] / stats["matches_played"]
        avg_GC_dict[player_id] = stats["total_GC"] / stats["matches_played"]

    return win_percentage_dict, avg_GS_dict, avg_GC_dict


In [82]:
win_percentage_dict, avg_GS_dict, avg_GC_dict = teamavgs_feature(training_dataset)
avg_GC_dict

{1: 6.578947368421052,
 4: 5.333333333333333,
 5: 5.666666666666667,
 7: 6.111111111111111,
 10: 7.0,
 11: 5.875,
 13: 6.375,
 2: 6.65,
 3: 7.294117647058823,
 8: 6.6,
 9: 6.823529411764706,
 15: 7.5625,
 17: 6.818181818181818,
 18: 7.461538461538462,
 14: 6.666666666666667,
 12: 7.470588235294118,
 16: 6.2,
 6: 7.076923076923077,
 20: 6.666666666666667,
 19: 7.0,
 21: 5.666666666666667,
 22: 6.6923076923076925,
 23: 6.666666666666667,
 24: 9.0,
 25: 6.5,
 27: 5.5,
 26: 6.555555555555555,
 28: 10.0,
 29: 8.0,
 30: 6.0,
 31: 6.0}

In [78]:
teamavgs_feature(training_dataset)
training_dataset['team1_win_percentage'] = training_dataset['team1_win_percentage'].astype('float64')
training_dataset['team2_win_percentage'] = training_dataset['team2_win_percentage'].astype('float64')
training_dataset['team1_avg_goals'] = training_dataset['team1_avg_goals'].astype('float64')
training_dataset['team2_avg_goals'] = training_dataset['team2_avg_goals'].astype('float64')
training_dataset['team1_avg_goalsconceded'] = training_dataset['team1_avg_goalsconceded'].astype('float64')
training_dataset['team2_avg_goalsconceded'] = training_dataset['team2_avg_goalsconceded'].astype('float64')
training_dataset.dtypes
training_dataset

,Match ID,Team 1 P1,Team 1 P2,Team 1 P3,Team 1 P4,Team 1 P5,Team 1 P6,Team 1 P7,Team 1 P8,Team 2 P1,...,Team 2 Goals,Team 1 Result,Team 2 Result,,team1_win_percentage,team2_win_percentage,team1_avg_goals,team2_avg_goals,team1_avg_goalsconceded,team2_avg_goalsconceded
0,1,1,4,5,7,10,11,13,0,2,...,5,1.0,0.0,,52.241019,48.062085,7.114150,6.583431,6.134294,7.029981
1,2,1,2,3,7,10,11,14,0,4,...,6,0.5,0.5,,48.123372,50.201171,7.106832,6.629243,6.596549,6.351967
2,3,1,2,4,8,9,13,16,0,3,...,5,1.0,0.0,,52.812915,45.174711,6.638123,6.901776,6.365830,6.642058
3,4,1,3,5,9,15,16,20,0,2,...,7,1.0,0.0,,49.607966,36.932773,6.654614,6.545693,6.684633,6.823179
4,5,1,2,3,6,12,15,21,0,4,...,5,1.0,0.0,,44.280809,52.873483,6.374061,6.918417,6.899963,6.223599
5,6,2,4,5,6,12,22,23,0,8,...,6,0.0,1.0,,37.354019,53.086325,6.157735,6.702562,6.508069,6.607744
6,7,1,2,5,6,11,20,22,0,3,...,4,1.0,0.0,,51.233693,46.678762,6.800795,6.472520,6.458073,6.902913
7,8,1,3,8,11,16,20,22,0,2,...,9,0.0,1.0,,54.002493,45.138146,6.731069,6.565518,6.558148,6.766638
8,9,1,3,7,12,13,15,24,0,2,...,9,0.0,1.0,,37.199002,50.513065,6.398954,6.884202,7.198895,6.699274
9,10,1,5,11,12,15,16,17,0,2,...,9,0.0,1.0,,49.148540,49.992099,6.690615,6.783016,6.595983,6.759328


In [72]:
player_ids = pd.read_csv("Player keys.csv")
player_id_dict = dict(zip(player_ids['player name'], player_ids['player_ID']))
player_id_dict

{'Sam': 1,
 'Kam': 2,
 'Rahul': 3,
 'Mo.O': 4,
 'Mo.E': 5,
 'Ollie.W': 6,
 'Jason': 7,
 'Waq': 8,
 'Kal': 9,
 'Omar': 10,
 'Jamie': 11,
 'Shyam': 12,
 'Gergo': 13,
 'Rob.B': 14,
 'Satpal': 15,
 'Saeed': 16,
 'Chris G': 17,
 'Saqi': 18,
 'Riz': 19,
 'Khalid': 20,
 'Yusuf': 21,
 'Jake': 22,
 'Mani': 23,
 'Max Nur': 24,
 'Abu': 25,
 'Paddy': 26,
 'Carlos': 27,
 'Faiz': 28,
 'Saj': 29,
 'Ashley': 30,
 'Has': 31,
 'Farooq': 32}

Creating a Test Match Set

In [76]:
# Define teams
team1_players = ['Waq', 'Jamie', 'Saeed', 'Sam', 'Jason', 'Yusuf', 'Carlos', 'Mo.O']
team2_players = ['Saqi', 'Jake', 'Satpal', 'Ashley', 'Shyam', 'Riz', 'Gergo', 'Saj']

# Create DataFrame for the match
match_data = pd.DataFrame(columns=['Match ID', 'Team 1 P1', 'Team 1 P2', 'Team 1 P3', 'Team 1 P4', 'Team 1 P5', 'Team 1 P6', 'Team 1 P7', 'Team 1 P8',
                                   'Team 2 P1', 'Team 2 P2', 'Team 2 P3', 'Team 2 P4', 'Team 2 P5', 'Team 2 P6', 'Team 2 P7', 'Team 2 P8',
                                   'Team 1 Goals', 'Team 2 Goals', 'Team 1 Result', 'Team 2 Result'])

# Assign players for Team 1
for i, player in enumerate(team1_players):
    match_data.at[0, f'Team 1 P{i+1}'] = player

# Assign players for Team 2
for i, player in enumerate(team2_players):
    match_data.at[0, f'Team 2 P{i+1}'] = player

match_data.fillna(0, inplace = True)

# Replace player names with player IDs
for column in match_data.columns:
    if column.startswith('Team 1 P') or column.startswith('Team 2 P'):
        match_data[column] = match_data[column].apply(lambda x: player_id_dict.get(x, x))

# Save DataFrame to CSV
match_data.to_csv('test_match.csv', index=False)

read_test_dataset = pd.read_csv("test_match.csv")
test_dataset = teamavgs_feature(read_test_dataset)
test_dataset

,Match ID,Team 1 P1,Team 1 P2,Team 1 P3,Team 1 P4,Team 1 P5,Team 1 P6,Team 1 P7,Team 1 P8,Team 2 P1,...,Team 1 Goals,Team 2 Goals,Team 1 Result,Team 2 Result,team1_win_percentage,team2_win_percentage,team1_avg_goals,team2_avg_goals,team1_avg_goalsconceded,team2_avg_goalsconceded
0,0,8,11,16,1,7,21,27,4,18,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


Training both ML models

In [74]:
clf = RandomForestClassifier()
reg_team1 = RandomForestRegressor()
reg_team2 = RandomForestRegressor()

def training_clf(training_dataset):
    df = training_dataset
    df = df.select_dtypes(exclude=['object']) 
    df["Team 1 Win"] = df["Team 1 Result"].apply(lambda x: 1 if x == 1 else 0)
    x_clf = df.drop(["Team 1 Result", "Team 2 Result", "Team 1 Win", "Team 1 Goals", "Team 2 Goals"], axis=1)
    y_clf = df["Team 1 Win"]
    clf.fit(x_clf, y_clf)
    return

def training_rfg(training_dataset):
    df = training_dataset
    df = df.select_dtypes(exclude=['object']) 
    x_reg = df.drop(["Team 1 Result", "Team 2 Result", "Team 1 Goals", "Team 2 Goals"], axis=1)
    y_reg_team1 = df["Team 1 Goals"]
    y_reg_team2 = df["Team 2 Goals"] 
    reg_team1.fit(x_reg, y_reg_team1)
    reg_team2.fit(x_reg, y_reg_team2)
    return


training_clf(training_dataset)
training_rfg(training_dataset)

Running both trained models on the test dataset

In [75]:
def predict_scores(test_dataset):
    df = test_dataset
    df = df.select_dtypes(exclude=['object'])
    x_test = df.drop(["Team 1 Result", "Team 2 Result", "Team 1 Goals", "Team 2 Goals"], axis=1)
    team1_goals = reg_team1.predict(x_test)
    team2_goals = reg_team2.predict(x_test)
    team1_win = clf.predict(x_test)
    return team1_goals, team2_goals, team1_win

predictions = predict_scores(test_dataset)
team1_goals, team2_goals, team1_win = predictions

# Print predicted goals
print("Predicted Goals:")
print("Team 1:", team1_goals)
print("Team 2:", team2_goals)
print("\nPredicted Results:")
if team1_win == 1:
    print("Team 1")
else:
    print("Team 2")

Predicted Goals:
Team 1: [9.44]
Team 2: [5.45]

Predicted Results:
Team 1
